In [1]:
# importa as bibliotecas que serão utilizadas.
import pandas as pd
import numpy as np
import json

In [2]:
path_main = '/home/jovyan/work/lucas/lucas-egc/estudo_caso'

##### Padronizando nomes dos órgaos da administração direta

In [3]:
# padroniza os nomes dos órgãos da administração direta 
# conforme dicionário json criado com as informações do
# notebook padronizacao_nomes_orgaos.ipynb

def ajustar_nome_orgao(valor: str) -> str:
    path_json = path_main +'/padronizacao_orgaos.json'
    with open(path_json, "r", encoding="utf-8") as f:
        orgaos_padronizados = json.load(f)

    for para, de in orgaos_padronizados.items():
        if valor in de:
            return para
            
    return valor
    

#### IACM

In [4]:
# carrega para o estudo de caso as informações do iacm
# armazenadas no arquivo csv gerado no notebook
# apendice_A-cumprimento_metas.ipynb
path_iacm = path_main + '/iacm_orgaos_anos_media.csv'
df_iacm = pd.read_csv(path_iacm)
df_iacm['ORGAO'] = df_iacm['ORGAO'].apply(ajustar_nome_orgao)
#df_iacm = df_iacm.set_index('ORGAO')
df_iacm.head()

,ORGAO,2021,2022,2023,2024,MEDIA
0,Secretaria Municipal de Mobilidade e Trânsito,100.0,50.60,77.73,39.67,67.0000
1,Secretaria Municipal de Inovação e Tecnologia,100.0,23.46,61.04,91.71,69.0525
2,Secretaria Municipal de Infraestrutura Urbana ...,100.0,64.73,77.94,51.50,73.5425
3,Secretaria Municipal de Cultura,100.0,66.67,60.00,72.58,74.8125
4,Secretaria Municipal de Segurança Urbana,0.0,100.00,100.00,100.00,75.0000


#### Rotatividade

In [5]:
# carrega para o estudo de caso as informações da rotatividade armazenadas 
# no arquivo csv gerado no notebook apendice_B-rotatividade.ipynb
path_rotatividade_pmsp = path_main + '/taxa_rotatividade_pmsp_anual.csv'
df_rotatividade_pmsp = pd.read_csv(path_rotatividade_pmsp)
#df_rotatividade_pmsp.head()

In [6]:
# carrega para o estudo de caso as informações da rotatividade armazenadas 
# no arquivo csv gerado no notebook apendice_B-rotatividade.ipynb
path_rotatividade = path_main + '/taxa_rotatividade_secret_subpref_anual.csv'
df_rotatividade = pd.read_csv(path_rotatividade)
df_rotatividade['ORGAO'] = df_rotatividade['SECRET_SUBPREF'].apply(ajustar_nome_orgao)
df_rotatividade = df_rotatividade[['ORGAO','ANO','TAXA_ROTATIVIDADE','TAXA_ROTATIVIDADE_COMISSAO']].merge(df_rotatividade_pmsp, on='ANO', how='inner', suffixes=('_ORGAO','_PMSP'))
df_rotatividade['TAXA_ROTATIVIDADE'] = df_rotatividade.apply(lambda x : 'Alta' if x.TAXA_ROTATIVIDADE_ORGAO > x.TAXA_ROTATIVIDADE_PMSP else 'Baixa', axis=1)
df_rotatividade['TAXA_ROTATIVIDADE_COMISSAO'] = df_rotatividade.apply(lambda x : 'Alta' if x.TAXA_ROTATIVIDADE_COMISSAO_ORGAO > x.TAXA_ROTATIVIDADE_COMISSAO_PMSP else 'Baixa', axis=1)
#df_rotatividade.head()

#### Efetivos em comissão

In [7]:
# carrega para o estudo de caso as informações da rotatividade armazenadas 
# no arquivo csv gerado no notebook apendice_B-rotatividade.ipynb
path_efetivos_comissao = path_main + '/percentual_efetivos_media_anual.csv'
df_efetivos_comissao = pd.read_csv(path_efetivos_comissao)
df_efetivos_comissao['ORGAO'] = df_efetivos_comissao['SECRET_SUBPREF'].apply(ajustar_nome_orgao)
df_efetivos_comissao = df_efetivos_comissao[['ORGAO','ANO','CDA_AGREGADOS','PERCENTUAL_EFETIVOS','PERCENTUAL_LEGAL']]
df_efetivos_comissao['SITUACAO_PERCENTUAL_EFETIVOS'] = df_efetivos_comissao.apply(lambda x : 'Acima' if x.PERCENTUAL_EFETIVOS >= x.PERCENTUAL_LEGAL else 'Abaixo', axis=1)
#df_efetivos_comissao.head()

## Correlações

In [8]:
# Supondo que df seja o seu DataFrame
df_somente_iacm = df_iacm.drop(columns=["MEDIA"])  # remove a coluna MEDIA

# faz a transformação
df_iacm_long = df_somente_iacm.melt(
    id_vars=["ORGAO"],          # mantém essa coluna fixa
    value_vars=["2021", "2022", "2023", "2024"],  # colunas que serão "derretidas"
    var_name="ANO",             # nome da nova coluna que terá os anos
    value_name="IACM"           # nome da coluna com os valores
)
df_iacm_long['ANO'] = df_iacm_long['ANO'].astype(int)

In [9]:
#df_orcamento[[col for col in df_orcamento.columns.to_list() if col != 'txtDescricaoOrgao']]
#df_orcamento_long = df_orcamento.drop(columns=['txtDescricaoOrgao'])
df_orcamento_long = df_orcamento.copy()

NameError: name 'df_orcamento' is not defined

In [ ]:
df_iacm_orcamento = df_orcamento_long.merge(df_iacm_long, on=['ORGAO','ANO'], how='inner')

In [ ]:
df_iacm_orcamento = df_iacm_orcamento[['IACM','taxa_atualizacao_orcamentaria', 'taxa_execucao_orcamentaria', 'taxa_liquidacao', 'taxa_pagamento', 'execucao_financeira_global']]
df_iacm_orcamento = df_iacm_orcamento.dropna(subset=['IACM'])

In [ ]:
!pip install matplotlib seaborn scipy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 4))
ax = axes[0]
sns.scatterplot(data=df_iacm_orcamento[['IACM','taxa_atualizacao_orcamentaria']], x='taxa_atualizacao_orcamentaria', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x taxa_atualizacao_orcamentaria')

ax = axes[1]
sns.scatterplot(data=df_iacm_orcamento[['IACM','taxa_execucao_orcamentaria']], x='taxa_execucao_orcamentaria', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x taxa_execucao_orcamentaria')

ax = axes[2]
sns.scatterplot(data=df_iacm_orcamento[['IACM','taxa_liquidacao']], x='taxa_liquidacao', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x taxa_liquidacao')

ax = axes[3]
sns.scatterplot(data=df_iacm_orcamento[['IACM','taxa_pagamento']], x='taxa_pagamento', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x taxa_pagamento')

ax = axes[4]
sns.scatterplot(data=df_iacm_orcamento[['IACM','execucao_financeira_global']], x='execucao_financeira_global', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x execucao_financeira_global')

plt.tight_layout()
plt.show()

In [ ]:
pearson = df_iacm_orcamento.corr(numeric_only=True, method="pearson")['IACM']
spearman = df_iacm_orcamento.corr(numeric_only=True, method="spearman")['IACM']
kendall = df_iacm_orcamento.corr(numeric_only=True, method="kendall")['IACM']
df_iacm_orcamento_corr = pd.DataFrame(pearson).rename(columns={'IACM':'Pearson'})
df_iacm_orcamento_corr = pd.concat([df_iacm_orcamento_corr, pd.DataFrame(spearman).rename(columns={'IACM':'Spearman'})], axis=1)
df_iacm_orcamento_corr = pd.concat([df_iacm_orcamento_corr, pd.DataFrame(kendall).rename(columns={'IACM':'Kendall'})], axis=1)
df_iacm_orcamento_corr

**Pearson**

Mede correlação linear.

* +1 → relação linear positiva perfeita
* -1 → relação linear negativa perfeita
* 0 → ausência de relação linear (mas pode existir relação não linear escondida).

**Spearman**

Mede correlação monotônica (baseada em ranks).

* +1 → quanto uma variável aumenta, a outra sempre aumenta (ordem perfeita).
* -1 → quanto uma variável aumenta, a outra sempre diminui (ordem inversa perfeita).
* 0 → ausência de relação monotônica (mas pode haver relação não monotônica).

**Kendall**

Mede concordância de pares (quantos pares de observações seguem a mesma ordem).

* +1 → todas as observações estão ordenadas da mesma forma nas duas variáveis.
* -1 → todas as observações estão ordenadas de forma oposta.
* 0 → não há relação de ordem clara.

**Conclusão:** Assim como em Pearson e Spearman, o IACM não apresenta correlações relevantes com os indicadores orçamentários.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau

# Função para calcular correlação + p-valor
def corr_and_pvalues(df, method="pearson"):
    cols = df.columns
    corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)

    for i in cols:
        for j in cols:
            if method == "pearson":
                corr, pval = pearsonr(df[i], df[j])
            elif method == "spearman":
                corr, pval = spearmanr(df[i], df[j])
            elif method == "kendall":
                corr, pval = kendalltau(df[i], df[j])
            else:
                raise ValueError("Método deve ser 'pearson', 'spearman' ou 'kendall'")
            
            corr_matrix.loc[i, j] = corr
            pval_matrix.loc[i, j] = pval

    return corr_matrix, pval_matrix

# Exemplo de uso
df = pd.DataFrame({
    "A": [1,2,3,4,5,6,7,8],
    "B": [2,4,5,4,6,7,8,9],
    "C": [8,7,6,5,4,3,2,1]
})

corr, pvals = corr_and_pvalues(df_iacm_orcamento, method="pearson")

print("=== Correlação ===")
print(corr.round(3))
print("\n=== P-valores ===")
print(pvals.round(4))


##### IACM x Rotatividade

In [ ]:
# Copia as informações da rotatividade para outra tabela
df_rotatividade_long = df_rotatividade.copy()

In [ ]:
# Junta as informações do IACM dos órgãos, ano a ano, com as taxas de rotatividade
df_iam_rotatividade = df_iacm_long.merge(df_rotatividade_long, on=['ORGAO','ANO'], how='inner')

In [ ]:
# Remove as colunas ORGAO e ANO e as linhas que não possuem o IACM definido
df_iam_rotatividade = df_iam_rotatividade[['IACM','TAXA_ROTATIVIDADE_ORGAO', 'TAXA_ROTATIVIDADE_COMISSAO_ORGAO']]
df_iam_rotatividade = df_iam_rotatividade.dropna(subset='IACM')

In [ ]:
# Exibi os gráficos com os valores do IACM e as taxas de rotatividade 
# (TAXA_ROTATIVIDADE_ORGAO e TAXA_ROTATIVIDADE_COMISSAO_ORGAO)
import matplotlib.pyplot as plt
import seaborn as sns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
ax = axes[0]
sns.scatterplot(data=df_iam_rotatividade[['IACM','TAXA_ROTATIVIDADE_ORGAO']], x='TAXA_ROTATIVIDADE_ORGAO', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 15)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x TAXA_ROTATIVIDADE_ORGAO')

ax = axes[1]
sns.scatterplot(data=df_iam_rotatividade[['IACM','TAXA_ROTATIVIDADE_COMISSAO_ORGAO']], x='TAXA_ROTATIVIDADE_COMISSAO_ORGAO', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 15)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x TAXA_ROTATIVIDADE_COMISSAO_ORGAO')

plt.tight_layout()
plt.savefig("figura_C1.png")
plt.show()

In [ ]:
# Cria a tabela com o cálculo da correlação estatística entre as variáveis
pearson = df_iam_rotatividade.corr(numeric_only=True, method="pearson")['IACM']
spearman = df_iam_rotatividade.corr(numeric_only=True, method="spearman")['IACM']
kendall = df_iam_rotatividade.corr(numeric_only=True, method="kendall")['IACM']
df_iam_rotatividade_corr = pd.DataFrame(pearson).rename(columns={'IACM':'Pearson'})
df_iam_rotatividade_corr = pd.concat([df_iam_rotatividade_corr, pd.DataFrame(spearman).rename(columns={'IACM':'Spearman'})], axis=1)
df_iam_rotatividade_corr = pd.concat([df_iam_rotatividade_corr, pd.DataFrame(kendall).rename(columns={'IACM':'Kendall'})], axis=1)
df_iam_rotatividade_corr.to_html('tabela_C1.html')
df_iam_rotatividade_corr

📌 **1) IACM vs TAXA_ROTATIVIDADE_ORGAO**

**✔ Pearson: 0.0437:**
Correlação positiva muito fraca. Praticamente nula. Não há relação linear significativa entre IACM e a rotatividade geral do órgão

**✔ Spearman: –0.1914:**
Correlação negativa fraca. Isso indica que, ao comparar apenas a ordem das observações, existe uma tendência fraca de que órgãos com maior rotatividade tendem a ter IACM um pouco menor — mas a relação é muito fraca.

**✔ Kendall: –0.1423:**
Confirma a leitura do Spearman. Há tendência fraca e negativa, mas não é forte o suficiente para indicar uma relação monotônica consistente.

🎯 **Conclusão desse par:**

✔ Não há correlação estatisticamente significativa.
✔ Pode haver uma leve tendência inversa, mas muito fraca.
✔ Nada que indique que o IACM varia em função clara da rotatividade geral.

📌 **2) IACM vs TAXA_ROTATIVIDADE_COMISSAO_ORGAO**

**✔ Pearson: 0.0431:**
Correlação positiva muito fraca. Irrelevante do ponto de vista estatístico

**✔ Spearman: –0.0461:**
Correlação negativa quase nula. Indica que a ordenação dos valores não segue padrão de aumento/diminuição entre as variáveis

**✔ Kendall: –0.0318:**
Confirma o Spearman: correlação praticamente inexistente

🎯 **Conclusão desse par:**

✔ Não há relação linear entre IACM e rotatividade de comissionados
✔ Também não há relação monotônica
✔ A variação no IACM não se explica pela rotatividade de comissionados


##### IACM x Efetivos em comissão

In [ ]:
# Estrutura a tabela com as informações dos percentuais de servidores 
# efetivos em cargos de comissão, mantendo a relação dos órgãos e separando
# os percentuais por níveis de cargos em colunas
df_efetivos_comissao_long = df_efetivos_comissao[['ORGAO','ANO','CDA_AGREGADOS','PERCENTUAL_EFETIVOS']].pivot(index=['ORGAO','ANO'], columns='CDA_AGREGADOS', values='PERCENTUAL_EFETIVOS').reset_index()

In [ ]:
# Junta as informações do IACM dos órgãos, ano a ano, com os
# percentuais dos servidores efetivos em cargos de comissão
df_iam_efetivos_comissao = df_iacm_long.merge(df_efetivos_comissao_long, on=['ORGAO','ANO'], how='inner')
df_iam_efetivos_comissao = df_iam_efetivos_comissao.drop(columns=['ANO','ORGAO'])

In [ ]:
# Exibi os gráficos com os valores do IACM e o percentual de servidores efetivos 
# em cargos de comissão ('CDA-1, CDA-2, CDA-3', 'CDA-4', 'CDA-5' e 'CDA-6')

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
ax = axes[0]
sns.scatterplot(data=df_iam_efetivos_comissao[['IACM','CDA-1, CDA-2, CDA-3']], x='CDA-1, CDA-2, CDA-3', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x CDA-1, CDA-2, CDA-3')

ax = axes[1]
sns.scatterplot(data=df_iam_efetivos_comissao[['IACM','CDA-4']], x='CDA-4', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x CDA-4')

ax = axes[2]
sns.scatterplot(data=df_iam_efetivos_comissao[['IACM','CDA-5']], x='CDA-5', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x CDA-5')

ax = axes[3]
sns.scatterplot(data=df_iam_efetivos_comissao[['IACM','CDA-6']], x='CDA-6', y='IACM', ax=ax, s=30, color='blue')
ax.set_xlim(0, 105)
ax.set_ylim(0, 105)
ax.set_title(f'IACM x CDA-6')

plt.tight_layout()
plt.savefig("figura_C2.png")
plt.show()

In [ ]:
# Cria a tabela com o cálculo da correlação estatística entre as variáveis
pearson = df_iam_efetivos_comissao.corr(numeric_only=True, method="pearson")['IACM']
spearman = df_iam_efetivos_comissao.corr(numeric_only=True, method="spearman")['IACM']
kendall = df_iam_efetivos_comissao.corr(numeric_only=True, method="kendall")['IACM']
df_iam_efetivos_comissao_corr = pd.DataFrame(pearson).rename(columns={'IACM':'Pearson'})
df_iam_efetivos_comissao_corr = pd.concat([df_iam_efetivos_comissao_corr, pd.DataFrame(spearman).rename(columns={'IACM':'Spearman'})], axis=1)
df_iam_efetivos_comissao_corr = pd.concat([df_iam_efetivos_comissao_corr, pd.DataFrame(kendall).rename(columns={'IACM':'Kendall'})], axis=1)
df_iam_efetivos_comissao_corr.to_html('tabela_C2.html')
df_iam_efetivos_comissao_corr

**⭐ Visão geral**

Todos os coeficientes estão entre **0.08 e 0.19**, o que significa:

✔ **Correlação positiva muito fraca**
✔ As variáveis **andam na mesma direção**, mas **a força da relação é baixa**
✔ Nada indica uma relação linear ou monotônica forte
✔ Ainda assim, existe um **sinal consistente**: todos os coeficientes são positivos → quando o CDA aumenta, o IACM tende a aumentar um pouco.

📌 **Interpretação detalhada variável por variável**
**1) IACM × (CDA-1, CDA-2, CDA-3)**

**Pearson: 0.1639** → relação linear **fraca**
**Spearman: 0.1096** → tendência monotônica **muito fraca**
**Kendall: 0.0878** → tendência ainda mais fraca

**✔ Conclusão**

Há uma **correlação positiva fraca** entre o IACM e o conjunto CDA-1/2/3.
O aumento desses indicadores parece acompanhar aumentos do IACM, mas a relação **não é forte** nem explicativa.

**2) IACM × CDA-4**

**Pearson: 0.1837** → correlação linear **fraca (ligeiramente maior)**
**Spearman: 0.1517** → tendência monotônica **fraca**
**Kendall: 0.1034** → mesma tendência

**✔ Conclusão**

Entre todas as variáveis, o **CDA-4 é um dos que mais se correlaciona com o IACM**, ainda que de forma fraca.
Existe uma leve associação: órgãos com melhor desempenho no CDA-4 tendem a ter IACM um pouco maior.

**3) IACM × CDA-5**

**Pearson: 0.1771** → correlação linear **fraca**
**Spearman: 0.1536** → tendência monotônica **fraca**
**Kendall: 0.1187** → reforça a tendência

**✔ Conclusão**

O CDA-5 tem correlação positiva e fraca com o IACM, semelhante ao CDA-4.
Há indício de que melhor desempenho no CDA-5 acompanha aumentos no IACM, mas sem força estatística suficiente para previsões.

**4) IACM × CDA-6**

**Pearson: 0.1928** → **maior correlação da tabela** (ainda assim fraca)
**Spearman: 0.1564** → tendência monotônica também **fraca**
**Kendall: 0.1336** → reforça que é o mais próximo de uma relação, mas ainda baixo

**✔ Conclusão**

O **CDA-6 é o que melhor se correlaciona com o IACM**, embora o valor ainda seja considerado **fraco** na escala de correlação.
Isso sugere que o CDA-6 influencia (ou acompanha) mais fortemente o IACM do que os demais indicadores.

**🟦 Síntese final**

Os resultados de correlação mostram que todas as variáveis CDA apresentam **correlação positiva e fraca** com o IACM.
Isso significa que:

Quando um CDA aumenta, o IACM tende a aumentar também
No entanto, a força da relação é baixa
As variáveis **não são boas preditoras individuais** do IACM
**CDA-6** apresenta a maior correlação, seguido de **CDA-4** e **CDA-5**
O conjunto **CDA-1,2,3** tem a menor correlação positiva

Em termos práticos:

👉 Há uma **associação consistente, porém fraca,** entre desempenho nos CDAs e IACM.
👉 O IACM **não depende fortemente** de nenhuma dessas variáveis analisadas isoladamente.
👉 Os CDAs podem contribuir para o IACM, mas somente como parte de um conjunto maior de fatores.

#### Preparando os dados para a padronização

In [ ]:
df_iacm_orcamento = df_iacm_long.merge(df_orcamento_long[['ORGAO','ANO','taxa_atualizacao_orcamentaria','taxa_execucao_orcamentaria','taxa_liquidacao','taxa_pagamento','execucao_financeira_global']], on=['ORGAO','ANO'], how='inner').dropna(subset='IACM')
df_iacm_orcamento_rotatividade = df_iacm_orcamento.merge(df_rotatividade_long[['ORGAO','ANO','TAXA_ROTATIVIDADE_ORGAO','TAXA_ROTATIVIDADE_COMISSAO_ORGAO']], on=['ORGAO','ANO'], how='inner')
df_iacm_orcamento_rotatividade_efetivos_comissao = df_iacm_orcamento_rotatividade.merge(df_efetivos_comissao_long[['ORGAO','ANO','CDA-1, CDA-2, CDA-3','CDA-4','CDA-5','CDA-6']], on=['ORGAO','ANO'], how='inner')
df_iacm_orcamento_rotatividade_efetivos_comissao = df_iacm_orcamento_rotatividade_efetivos_comissao.fillna(0)

In [ ]:
#df_iacm_orcamento_rotatividade.head()

In [ ]:
#df_iacm_orcamento_rotatividade_efetivos_comissao.tail()

In [ ]:
df_iacm_orcamento_rotatividade_efetivos_comissao.describe()

#### Padronizando os dados

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
cols = ["IACM", "TAXA_ROTATIVIDADE_ORGAO", "TAXA_ROTATIVIDADE_COMISSAO_ORGAO",
        "CDA-1, CDA-2, CDA-3", "CDA-4", "CDA-5", "CDA-6"]

#cols = ["IACM"]

X = df_iacm_orcamento_rotatividade_efetivos_comissao[cols].copy()

# Dá peso extra ao IACM (exemplo: multiplica por 10)
X["IACM"] = X["IACM"] * 10

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#### Escolhendo o número de clusters

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

inertia = []
K = range(1, 50)
for k in K:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(K, inertia, marker="o")
plt.xlabel("Número de clusters")
plt.ylabel("Inércia (Soma dos quadrados intra-cluster)")
plt.title("Método do Cotovelo (elbow method)")
plt.show()


#### Criando os clusters

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_iacm_orcamento_rotatividade_efetivos_comissao["Cluster"] = kmeans.fit_predict(X_scaled)


In [ ]:
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "notebook"

df = df_iacm_orcamento_rotatividade_efetivos_comissao[['Cluster','IACM']].copy()
df['Cluster'] = df['Cluster'].astype(int)

# supondo que df tenha as colunas Cluster e IACM
fig = px.box(
    df_iacm_orcamento_rotatividade_efetivos_comissao,
    x="Cluster",
    y="IACM",
    color="Cluster",      # cores diferentes por cluster
    points="all",         # mostra os pontos individuais além do boxplot
    title="Distribuição do IACM nos clusters",
    category_orders={"Cluster": sorted(df["Cluster"].unique())}
)

# Layout mais limpo
fig.update_layout(
    xaxis_title="Cluster",
    yaxis_title="IACM",
    template="plotly_white"
)

fig.show()


In [ ]:
cluster_summary = df_iacm_orcamento_rotatividade_efetivos_comissao.groupby("Cluster").agg(IACM_min=('IACM','min'), IACM_max=('IACM','max'), QTD=('ORGAO','count'))
cluster_summary

In [ ]:
cluster_summary = df_iacm_orcamento_rotatividade_efetivos_comissao.groupby("Cluster")[['IACM','taxa_atualizacao_orcamentaria',
                                                                                       'taxa_execucao_orcamentaria','taxa_liquidacao',
                                                                                       'taxa_pagamento','execucao_financeira_global']].mean()
cluster_summary

In [ ]:
cluster_summary = df_iacm_orcamento_rotatividade_efetivos_comissao.groupby("Cluster")[['IACM','TAXA_ROTATIVIDADE_ORGAO','TAXA_ROTATIVIDADE_COMISSAO_ORGAO']].mean()
cluster_summary

In [ ]:
cluster_summary = df_iacm_orcamento_rotatividade_efetivos_comissao.groupby("Cluster")[['IACM','CDA-1, CDA-2, CDA-3','CDA-4','CDA-5','CDA-6']].mean()
cluster_summary

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# PCA
pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

df_plot = df_iacm_orcamento_rotatividade_efetivos_comissao.copy()
df_plot["PC1"] = components[:, 0]
df_plot["PC2"] = components[:, 1]

cores = {
    0: "#e41a1c",  # vermelho
    1: "#377eb8",  # azul
    2: "#4daf4a",  # verde
    3: "#984ea3",  # roxo
    4: "#ff7f00"   # laranja
}
plt.figure(figsize=(12, 8))

# Plota cada cluster com cor única
for cluster, dados in df_plot.groupby("Cluster"):
    plt.scatter(
        dados["PC1"], 
        dados["PC2"], 
        label=f"Cluster {cluster}", 
        color=cores[cluster], 
        alpha=0.7
    )
    # Adiciona rótulos (Órgão + Ano)
    for i, row in dados.iterrows():
        
        if row["Cluster"] not in (3,4):
            continue
            
        plt.text(
            row["PC1"]+0.1, row["PC2"]+0.1,  # desloca p/ não sobrepor o ponto
            f'{row["ORGAO"].replace("Secretaria Municipal ","").replace("de ","").replace("das ","")} ({row["ANO"]})',
            fontsize=8
        )

plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.title("Clusters de órgãos (PCA)")
plt.legend(title="Clusters")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

plt.scatter(components[:,0], components[:,1], c=df_iacm_orcamento_rotatividade_efetivos_comissao["Cluster"], cmap="viridis")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Clusters de órgãos/setores")
plt.colorbar(label="Cluster")
plt.show()
